## Engenheiro de Dados Pleno: Teste Técnico

### Candidato: Luciano Martins dos Santos

### O objetivo é extrair apenas dos dados necessários de todo o conjunto de arquivos json, para ganhar de performance nas consultas finais:
### 1 - Top 10 Condições
### 2 - Top 10 Prescrições de Medicamentos
### 3 - Quantidade de gêneros masculinos

In [1]:
import pandas as pd
import os
import glob
import psycopg2

In [2]:
path = "C:\\Users\\lucia\\Documents\\teste_engenheiro-main\\teste_engenheiro-main\\data"

In [3]:
# a função glob faz com que eu consiga carregar todos os arquivos json do path de uma só vez para uma variável
json_files = glob.glob(os.path.join(path, '*.json'))

In [4]:
# esta estrutura, método concat combinado com a List Comprehension, faz com que eu transforme todo conteúdo num único dataframe. 
pacientes_source =  pd.concat([pd.read_json(file) for file in json_files], ignore_index=True)

In [106]:
# Abre conexão com o PosrgresSQL. Pode ser reproduzido facilmente em qualuer instalação do PgAdmin On-Premisse
conn = psycopg2.connect(host='localhost', database='postgres', user='postgres', password='admin')
cur = conn.cursor()

In [88]:
# Carrega a tabela do patients no PostgresSQL
for i in range(len(pacientes_source)):
    if pacientes_source.iloc[i].entry.get('resource').get('resourceType') == 'Patient':
        cur.execute("""
            INSERT INTO patients (resource_type, gender)
            VALUES (%s, %s)
        """, (
            pacientes_source.iloc[i].entry.get('resource').get('resourceType'),
            pacientes_source.iloc[i].entry.get('resource').get('gender')
        ))
conn.commit()

In [99]:
# Carrega a tabela do conditions no PostgresSQL
for i in range(len(pacientes_source)):
    if pacientes_source.iloc[i].entry.get('resource').get('resourceType') == 'Condition':
        cur.execute("""
            INSERT INTO conditions (resource_type, condition)
            VALUES (%s, %s)
        """, (
            pacientes_source.iloc[i].entry.get('resource').get('resourceType'),
            pacientes_source.iloc[i].entry.get('resource').get('code').get('text')
        ))
conn.commit()

In [107]:
# Carrega a tabela do medications no PostgresSQL
for i in range(len(pacientes_source)):
    if pacientes_source.iloc[i].entry.get('resource').get('resourceType') == 'MedicationRequest':
        cur.execute("""
            INSERT INTO medications (resource_type, medication)
            VALUES (%s, %s)
        """, (
            pacientes_source.iloc[i].entry.get('resource').get('resourceType'),
            pacientes_source.iloc[i].entry.get('resource').get('medicationCodeableConcept').get('text')
        ))
conn.commit()

In [105]:
# Fecha conexão com o PostgreSQL
conn.close()